<a href="https://colab.research.google.com/github/zHazyl/ml-from-scratch/blob/main/multi-agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
from langchain.chat_models import ChatOpenAI
from google.colab import userdata
import os

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

llm = ChatOpenAI()

In [ ]:
!pip install langchain-google-genai

In [30]:
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 4.8 MB/s eta 0:00:00


In [24]:
from langchain.vectorstores import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

vector_store = Chroma(
    persist_directory="emb",
    embedding_function=embeddings
)

def build_retriever(k=1):
    return vector_store.as_retriever(
        k=k
    )

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)

loader = PyPDFLoader("/content/spice.pdf")
docs = loader.load_and_split(text_splitter)

vector_store.add_documents(docs)

In [25]:
retriever = build_retriever()

In [37]:
input = "Which countries produce spice most?"
questions = llm.invoke(f"""
You're a good assistant that support me ask questions with different perspective to answer my question better.
For example:

Question: How can I open a company?
Answer: Which laws you need to know?++What skills you need to prepare?++Which procedure you need to make?

Question: How can I make udon?
Answer: What are ingredients to make udon?++Where to find ingredients to make udon?++What recipe to make udon?
=========
Question: {input}
Answer:
""").content.split('++')


In [45]:
from langchain.chains import RetrievalQA

map = RetrievalQA.from_llm(
    llm=llm,
    retriever=retriever,
)

In [47]:
ans1 = map.invoke(questions[0])
ans1

{'query': 'What are the top exporting countries of spices?',
 'result': 'The top spice-producing countries based on the information provided are:\n\n1. India\n2. Bangladesh\n3. Turkey\n4. China\n5. Pakistan\n6. Iran\n7. Nepal\n8. Colombia\n9. Ethiopia\n10. Sri Lanka'}

In [48]:
ans2 = map.invoke(questions[1])
ans2

{'query': 'Which countries have the highest spice production?',
 'result': 'The countries with the highest spice production based on the provided data are:\n\n1. India\n2. Bangladesh\n3. Turkey'}

In [49]:
ans3 = map.invoke(questions[2])
ans3

{'query': 'How do different countries contribute to the global spice market?',
 'result': 'India is a major contributor to the global spice market, accounting for 75% of global spice production. Other significant spice-producing countries include Bangladesh, Turkey, China, Pakistan, Iran, Nepal, Colombia, Ethiopia, and Sri Lanka. These countries also play a role in shaping the global spice market, with each contributing different types of spices and herbs to the industry.'}

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)

loader = PyPDFLoader("/content/Spring.in.Action.6th.Edition.Craig.Walls.Manning.9781617297571.EBooksWorld.ir.pdf")
docs = loader.load_and_split(text_splitter)

vector_store.add_documents(docs)

In [143]:
from threading import Thread
from langchain_google_genai import ChatGoogleGenerativeAI

def answer_question(input):
  questions_prompt = f"""
  You're a good assistant that support me ask questions with different perspective to answer my question better.
  For example:

  Question: How can I open a company?
  Answer: Which laws you need to know?++What skills you need to prepare?++Which procedure you need to make?

  Question: How can I make udon?
  Answer: What are ingredients to make udon?++Where to find ingredients to make udon?++What recipe to make udon?
  =========
  Question: {input}
  Answer:
  """

  gpt = RetrievalQA.from_llm(
    llm=ChatOpenAI(),
    retriever=retriever
  )
  gemini = RetrievalQA.from_llm(
    llm=ChatGoogleGenerativeAI(model="gemini-pro", convert_system_message_to_human=True),
    retriever=retriever
  )

  llm = ChatGoogleGenerativeAI(model="gemini-pro")

  llms = [None, gpt, gemini]
  index = -1

  questions = llm.invoke(questions_prompt).content
  questions = list(filter(lambda q: len(q) != 0, questions.split("++")))
  questions.append(input)
  print(questions)

  ans = []

  for question in questions:
    t = Thread(target=lambda q: ans.append(llms[index].invoke(q)['result']), args=[question])
    t.start()
    index*=-1

  while True:
    if len(ans) == len(questions):
      return '\n-'.join(ans)


In [144]:
input = "How to build Spring app"

context = answer_question(input)

final = f"""
Answer question based on following context:
- {context}

Question: {input}
Answer:
"""

final_ans = llm.invoke(final)

['What tools are needed to build a Spring app?', 'What skillset is needed to build a Spring app?', 'What are the steps to build a Spring app?', 'How to build Spring app']


In [145]:
final_ans.content

'To build a Spring app, you can initialize a Spring application using Spring Initializr or use an IDE like NetBeans or Spring Tool Suite. Then, add dependencies, configure settings, and write your application logic using Spring, Spring Boot, and related libraries and frameworks. Knowledge of Java programming, dependency injection, web development, database integration, and Spring Boot would be beneficial for building a Spring application. Familiarity with web technologies like HTML, CSS, and JavaScript might also be needed depending on the type of application you are creating.'